### Adding dataframes for future analyzation

In [1]:
import pandas as pd

order_list = pd.read_csv('data/List of Orders.csv')
order_details = pd.read_csv('data/Order Details.csv')
sales = pd.read_csv('data/Sales target.csv')

##### Order list

In [2]:
print(order_list.info())
order_list.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      500 non-null    object
 1   Order Date    500 non-null    object
 2   CustomerName  500 non-null    object
 3   State         500 non-null    object
 4   City          500 non-null    object
dtypes: object(5)
memory usage: 22.0+ KB
None


,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


##### Order details

In [3]:
print(order_details.info())
order_details.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      1500 non-null   object 
 1   Amount        1500 non-null   float64
 2   Profit        1500 non-null   float64
 3   Quantity      1500 non-null   int64  
 4   Category      1500 non-null   object 
 5   Sub-Category  1500 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 70.4+ KB
None


,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones


##### Sales


In [4]:
print(sales.info())
sales.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Month of Order Date  36 non-null     object 
 1   Category             36 non-null     object 
 2   Target               36 non-null     float64
dtypes: float64(1), object(2)
memory usage: 992.0+ bytes
None


,Month of Order Date,Category,Target
0,Apr-18,Furniture,10400.0
1,May-18,Furniture,10500.0
2,Jun-18,Furniture,10600.0
3,Jul-18,Furniture,10800.0
4,Aug-18,Furniture,10900.0


##### Join our dataframes


In [5]:
orders = pd.merge(order_list, order_details, on='Order ID')
orders.head()

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,66.0,-12.0,5,Clothing,Stole
2,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,01-04-2018,Pearl,Maharashtra,Pune,168.0,-111.0,2,Electronics,Phones


##### Missing values

In [6]:
orders.isna().sum()

Order ID        0
Order Date      0
CustomerName    0
State           0
City            0
Amount          0
Profit          0
Quantity        0
Category        0
Sub-Category    0
dtype: int64

##### Duplicates

In [7]:
orders.duplicated().sum()

0

##### Data Cleaning
1. Devide order date into columns
2. Delete Order ID column

In [8]:
orders[['Day', 'Month', 'Year']] = orders['Order Date'].str.split('-', expand=True)
orders['Month'] = orders['Month'].apply(int)
orders['Day'] = orders['Day'].apply(int)
orders['Year'] = orders['Year'].apply(int)
orders.drop(['Order Date', 'Order ID'], inplace=True, axis=1)
orders.head()

,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category,Day,Month,Year
0,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases,1,4,2018
1,Bharat,Gujarat,Ahmedabad,66.0,-12.0,5,Clothing,Stole,1,4,2018
2,Bharat,Gujarat,Ahmedabad,8.0,-2.0,3,Clothing,Hankerchief,1,4,2018
3,Bharat,Gujarat,Ahmedabad,80.0,-56.0,4,Electronics,Electronic Games,1,4,2018
4,Pearl,Maharashtra,Pune,168.0,-111.0,2,Electronics,Phones,1,4,2018


In [9]:
from time import strptime
sales[['Month', 'Year']] = sales['Month of Order Date'].str.split('-', expand=True)
sales['Month'] = sales['Month'].apply(lambda x: strptime(x, '%b').tm_mon)
sales['Year'] = sales['Year'].apply(lambda x: int(f"20{x}"))
sales.drop('Month of Order Date', axis=1, inplace=True)
sales.head()

,Category,Target,Month,Year
0,Furniture,10400.0,4,2018
1,Furniture,10500.0,5,2018
2,Furniture,10600.0,6,2018
3,Furniture,10800.0,7,2018
4,Furniture,10900.0,8,2018


##### Joining the sales to order frame


In [10]:
df = pd.merge(sales, orders, on=['Month', 'Year'])
df.head()

,Category_x,Target,Month,Year,CustomerName,State,City,Amount,Profit,Quantity,Category_y,Sub-Category,Day
0,Furniture,10400.0,4,2018,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases,1
1,Furniture,10400.0,4,2018,Bharat,Gujarat,Ahmedabad,66.0,-12.0,5,Clothing,Stole,1
2,Furniture,10400.0,4,2018,Bharat,Gujarat,Ahmedabad,8.0,-2.0,3,Clothing,Hankerchief,1
3,Furniture,10400.0,4,2018,Bharat,Gujarat,Ahmedabad,80.0,-56.0,4,Electronics,Electronic Games,1
4,Furniture,10400.0,4,2018,Pearl,Maharashtra,Pune,168.0,-111.0,2,Electronics,Phones,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4500 entries, 0 to 4499
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Category_x    4500 non-null   object 
 1   Target        4500 non-null   float64
 2   Month         4500 non-null   int64  
 3   Year          4500 non-null   int64  
 4   CustomerName  4500 non-null   object 
 5   State         4500 non-null   object 
 6   City          4500 non-null   object 
 7   Amount        4500 non-null   float64
 8   Profit        4500 non-null   float64
 9   Quantity      4500 non-null   int64  
 10  Category_y    4500 non-null   object 
 11  Sub-Category  4500 non-null   object 
 12  Day           4500 non-null   int64  
dtypes: float64(3), int64(4), object(6)
memory usage: 492.2+ KB


In [12]:
df.to_excel('data/CleanedDate.xlsx')
